In [1]:
import pandas as pd
import folium
import geopandas as gpd
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

In [2]:
school = pd.read_csv("../data/curated/school_df_21_preprocessed.csv")
mall = pd.read_csv("../data/raw/mall_info.csv")
train = pd.read_excel("../data/raw/train_station_20to21.xlsx")
postcode_SA2 = pd.read_csv("../data/raw/postcode_info.csv")
population = pd.read_csv("../data/curated/population_postcode_SA2.csv")
rent_income = pd.read_csv("../data/curated/rent_income21.csv")

# School

In [3]:
school

,Unnamed: 0,Education_Sector,School_Name,School_Type,Address_Line_1,Address_Town,Address_Postcode,LGA_Name,X,Y
0,0,Government,Alberton Primary School,Primary,21 Thomson Street,Alberton,3971,Wellington (S),146.666601,-38.617713
1,1,Government,Allansford and District Primary School,Primary,Frank Street,Allansford,3277,Warrnambool (C),142.590393,-38.386281
2,2,Government,Avoca Primary School,Primary,118 Barnett Street,Avoca,3467,Pyrenees (S),143.475649,-37.084502
3,3,Government,Avenel Primary School,Primary,40 Anderson Street,Avenel,3664,Strathbogie (S),145.234722,-36.901368
4,4,Government,Warrandyte Primary School,Primary,5-11 Forbes Street,Warrandyte,3113,Manningham (C),145.213980,-37.742675
...,...,...,...,...,...,...,...,...,...,...
2286,2286,Independent,Youth2Industry College,Secondary,"Level 2, 220 Albert Road",SOUTH MELBOURNE,3205,Port Phillip (C),144.966417,-37.837469
2287,2287,Catholic,St Angela of the Cross Primary School,Primary,181 Mills Road,WARRAGUL,3820,Baw Baw (S),145.961611,-38.141528
2288,2288,Independent,My College,Primary,92 – 96 Railway Crescent,DALLAS,3047,Hume (C),144.923692,-37.675751
2289,2289,Independent,Woodline Primary,Primary,405 Barrabool Road,CERES,3221,Greater Geelong (C),144.285354,-38.171776


In [4]:
sch_lat = school["Y"].to_list()
sch_lng = school["X"].to_list()
sch_type = school["School_Type"].to_list()

In [5]:
sch_point = []
for i in range(len(sch_lat)):
    sch_point.append((sch_lat[i], sch_lng[i], sch_type[i]))

In [6]:
#sch_point

In [7]:
poa_sch = []
for i in school["Address_Postcode"]:
    poa_sch.append(i)

poa_school = []
for j in poa_sch:
    if j not in poa_school:
        poa_school.append(j)

len(poa_school)

580

In [8]:
num_school = []
for poa in poa_school: 
    num = 0
    for index,row in school.iterrows():
        if row['Address_Postcode'] == poa:
            num += 1
    
    num_school.append((poa,num))
    
#num_school

In [9]:
len(num_school)

580

# Mall

In [10]:
mall

,Unnamed: 0,Mall,Stores,State,City,Latitude,Longitude
0,0,206 Bourke Street,16.0,Victoria,Melbourne,-37.812733,144.966947
1,1,Acland Court Shopping Centre,13.0,Victoria,St Kilda,-37.868967,144.980617
2,2,Altona Gate Shopping Centre,83.0,Victoria,Melbourne,-37.828989,144.846270
3,3,Balnarring Village Shopping Centre,29.0,Victoria,Balnarring,-38.373199,145.125010
4,4,Barkly Square,46.0,Victoria,Brunswick,-37.775695,144.962004
...,...,...,...,...,...,...,...
186,186,Wodonga Plaza,48.0,Victoria,Wodonga,-36.121153,146.881917
187,187,Woodgrove Shopping Centre,161.0,Victoria,Melton,-37.686167,144.561270
188,188,Woolworths Whitebox Rise Wodonga,8.0,Victoria,Wodonga,-36.139008,146.892456
189,189,Wyndham Vale Square Shopping Centre,11.0,Victoria,Wyndham Vale,-37.888057,144.607036


In [11]:
mall_lat = mall["Latitude"].to_list()
mall_lng = mall["Longitude"].to_list()
mall_store = mall["Stores"].to_list()

In [12]:
mall_point = []
for i in range(len(mall_lat)):
    mall_point.append((mall_lat[i], mall_lng[i], mall_store[i]))

In [13]:
#mall_point

# Train station

In [14]:
train

,Fin_year,Stop_ID,Stop_name,Stop_lat,Stop_long,Pax_annual
0,FY20-21,19980,Melton Railway Station (Melton South),-37.703362,144.572217,326100
1,FY20-21,19981,Rockbank Railway Station (Rockbank),-37.729258,144.650630,57900
2,FY20-21,19982,Deer Park Railway Station (Deer Park),-37.777760,144.772307,240650
3,FY20-21,20020,Ardeer Railway Station (Ardeer),-37.783063,144.802193,23050
4,FY20-21,20287,Albury Railway Station (Albury (NSW)),-36.084263,146.924518,9950
...,...,...,...,...,...,...
86,FY20-21,47647,Wyndham Vale Railway Station (Manor Lakes),-37.872884,144.608734,358050
87,FY20-21,47648,Tarneit Railway Station (Tarneit),-37.832169,144.694716,593000
88,FY20-21,48804,Cobblebank Railway Station (Cobblebank),-37.712542,144.604103,46950
89,FY20-21,52011,Caroline Springs Railway Station (Ravenhall),-37.766144,144.735748,64400


In [15]:
train_lat = train["Stop_lat"].to_list()
train_lng = train["Stop_long"].to_list()
train_id = train["Stop_ID"].to_list()

In [16]:
train_point = []
for i in range(len(train_lat)):
    train_point.append((train_lat[i], train_lng[i], train_id[i]))

# Population

In [17]:
population

,Unnamed: 0,Postcode,Longitude,Latitude,SA2 name,S/T name,2001,2002,2003,2004,...,change 2011-2021 number,change 2011-2021 %,Area km2,population density 2021(persons/km2),average change %,2022,2023,2024,2025,2026
0,0,3002,144.982207,-37.818517,East Melbourne,Victoria,3731,3859,4243,4460,...,-276,-0.053,2.9,1711.2,1.014927,5036.0,5111.0,5187.0,5264.0,5343.0
1,1,3005,144.950858,-37.824608,Docklands,Victoria,154,926,1913,3089,...,9746,1.573,2.4,6522.9,1.422278,22674.0,32249.0,45867.0,65236.0,92784.0
2,2,3008,144.948039,-37.814719,Docklands,Victoria,154,926,1913,3089,...,9746,1.573,2.4,6522.9,1.422278,22674.0,32249.0,45867.0,65236.0,92784.0
3,3,3010,144.961351,-37.796152,Parkville,Victoria,5332,5327,5253,5201,...,895,0.137,4.0,1838.9,1.017541,7577.0,7710.0,7845.0,7983.0,8123.0
4,4,3050,144.954581,-37.800324,Parkville,Victoria,5332,5327,5253,5201,...,895,0.137,4.0,1838.9,1.017541,7577.0,7710.0,7845.0,7983.0,8123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,587,3990,145.501279,-38.455399,Wonthaggi - Inverloch,Victoria,17647,17801,17956,18117,...,6327,0.310,728.8,36.7,1.021019,27285.0,27859.0,28445.0,29043.0,29653.0
588,588,3991,145.466781,-38.483041,Wonthaggi - Inverloch,Victoria,17647,17801,17956,18117,...,6327,0.310,728.8,36.7,1.021019,27285.0,27859.0,28445.0,29043.0,29653.0
589,589,3992,145.534315,-38.569135,Wonthaggi - Inverloch,Victoria,17647,17801,17956,18117,...,6327,0.310,728.8,36.7,1.021019,27285.0,27859.0,28445.0,29043.0,29653.0
590,590,3995,145.630713,-38.578071,Wonthaggi - Inverloch,Victoria,17647,17801,17956,18117,...,6327,0.310,728.8,36.7,1.021019,27285.0,27859.0,28445.0,29043.0,29653.0


In [18]:
pop_lat = population["Latitude"].to_list()
pop_lng = population["Longitude"].to_list()
pop11_21 = population["change 2011-2021 %"].to_list()

In [19]:
pop_point = []
for i in range(len(pop_lat)):
    pop_point.append((pop_lat[i], pop_lng[i], pop11_21[i]))

In [20]:
pop_point

[(-37.818517, 144.982207, -0.053),
 (-37.824608, 144.950858, 1.5730000000000002),
 (-37.814719, 144.948039, 1.5730000000000002),
 (-37.796152, 144.961351, 0.1369999999999999),
 (-37.800324, 144.954581, 0.1369999999999999),
 (-37.788996, 144.948903, 0.1369999999999999),
 (-37.807101, 144.907953, 0.242),
 (-37.807135, 144.861162, 0.072),
 (-37.814183, 144.888686, 0.093),
 (-37.838242, 144.880556, 0.13),
 (-37.863743, 144.888461, 0.047),
 (-37.86172, 144.812736, 0.114),
 (-37.784683, 144.854014, 0.158),
 (-37.777204, 144.829945, 0.0279999999999999),
 (-37.795937, 144.793961, 0.0279999999999999),
 (-37.743361, 144.796693, -0.016),
 (-37.78272, 144.770641, 0.303),
 (-37.75045631, 144.649936, 5.434),
 (-37.719, 144.6684, 5.434),
 (-37.835562, 144.839704, 0.113),
 (-37.932419, 144.754147, 1.084),
 (-37.875066, 144.777165, -0.046),
 (-37.837165, 144.705831, -0.048),
 (-37.79119, 144.923403, -0.037),
 (-37.774486, 144.897221, 0.086),
 (-37.74107, 144.857638, -0.004),
 (-37.705324, 144.82246, -0

# Rent and Income

In [21]:
rent_income

,Unnamed: 0,Postcode,Median Yearly rent price in 2021,Median income in 2021-2022,affordable index
0,0,3000,19240,28312.07,1.471521
1,1,3003,19240,20105.80,1.045000
2,2,3005,19240,55608.20,2.890239
3,3,3008,19240,55608.20,2.890239
4,4,3010,19240,54240.67,2.819162
...,...,...,...,...,...
663,663,3922,18200,42006.06,2.308025
664,664,3923,18200,42006.06,2.308025
665,665,3925,18200,42006.06,2.308025
666,666,3991,18200,42818.06,2.352641


In [22]:
rent_income["Postcode"] = rent_income["Postcode"].astype(str)

In [23]:
postcode = pd.DataFrame(postcode_SA2, columns=["Postcode", "Longitude", "Latitude"])
postcode

,Postcode,Longitude,Latitude
0,3000,144.956776,-37.817403
1,3001,144.956776,-37.817403
2,3002,144.982207,-37.818517
3,3003,144.949592,-37.810871
4,3004,144.970161,-37.844246
...,...,...,...
3529,8111,144.970161,-37.844246
3530,8120,144.970161,-37.844246
3531,8205,144.970161,-37.844246
3532,8785,145.208504,-38.016114


In [24]:
#rent_income_merge = pd.DataFrame(pd.merge(rent_income, postcode, on='Postcode', how='inner'))
#rent_income_merge.dropna(axis='rows')

In [25]:
#rent_income_new = rent_income_merge.drop_duplicates(["Postcode"])

In [26]:
#re_in_lat = rent_income_new["Latitude"].to_list()
#re_in_lng = rent_income_new["Longitude"].to_list()
#re_in_aff = rent_income_new["affordable index"].to_list()

In [27]:
#re_in_point = []
#for i in range(len(re_in_lat)):
#    re_in_point.append((re_in_lat[i], re_in_lng[i], re_in_aff[i]))

# Map

In [28]:
#postcode_SA2

In [29]:
sf = gpd.read_file("../data/POA_2021_AUST_GDA94_SHP/POA_2021_AUST_GDA94.shp")
sf = sf.rename(columns={'POA_CODE21':'Postcode'})

In [30]:
sf

,Postcode,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
0,0800,0800,AUS,Australia,3.1731,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.081893,0.000264,"POLYGON ((130.83680 -12.45378, 130.83683 -12.4..."
1,0810,0810,AUS,Australia,24.4283,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.241859,0.002031,"POLYGON ((130.89985 -12.36568, 130.89874 -12.3..."
2,0812,0812,AUS,Australia,35.8899,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.278789,0.002983,"POLYGON ((130.91914 -12.40788, 130.91830 -12.4..."
3,0820,0820,AUS,Australia,39.0642,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.409134,0.003248,"POLYGON ((130.85260 -12.43995, 130.85088 -12.4..."
4,0822,0822,AUS,Australia,150775.8030,http://linked.data.gov.au/dataset/asgsed3/POA/...,90.601833,12.564239,"MULTIPOLYGON (((136.56647 -12.08393, 136.56703..."
...,...,...,...,...,...,...,...,...,...
2639,7469,7469,AUS,Australia,5893.5125,http://linked.data.gov.au/dataset/asgsed3/POA/...,11.341556,0.636169,"MULTIPOLYGON (((144.79215 -41.46479, 144.79211..."
2640,7470,7470,AUS,Australia,108.2441,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.781048,0.011725,"POLYGON ((145.57768 -41.78641, 145.57686 -41.7..."
2641,9494,No usual address (Aust.),AUS,Australia,0.0000,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.000000,0.000000,None
2642,9797,Migratory - Offshore - Shipping (Aust.),AUS,Australia,0.0000,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.000000,0.000000,None


In [31]:
poa_vic = postcode_SA2["Postcode"].to_list()
poa_vic = set(poa_vic)

In [32]:
sf_new = pd.DataFrame()

for poa in poa_vic:
    df = sf[sf["Postcode"] == str(poa)]
    sf_new = pd.concat([sf_new, df])

In [33]:
sf_new['geometry'] = sf_new['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [34]:
geoJSON = sf_new[['Postcode','geometry']].drop_duplicates('Postcode').to_json()

In [35]:
re_in = gpd.GeoDataFrame(pd.merge(rent_income[['Postcode', 'affordable index']], sf, on='Postcode', how='inner'))
re_inGeo = re_in[['Postcode','geometry']].drop_duplicates('Postcode').to_json()
print(re_inGeo[:300])

{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"Postcode": "3000"}, "geometry": {"type": "Polygon", "coordinates": [[[144.96138273000008, -37.82051446999998], [144.96062273000007, -37.820754489999956], [144.96000274000005, -37.82104448999996], [144.959030740


In [36]:
#m = folium.Map(location=[-36, 145.3906], tiles="Stamen Terrain", zoom_start=7)

In [43]:
poagroup = folium.FeatureGroup(name='Postcode_Layer', control=True)

popgroup = folium.FeatureGroup(name='Population_Layer', control=True)

schgroup = folium.FeatureGroup(name='School_Layer', control=True)

mallgroup = folium.FeatureGroup(name='Mall_Layer', control=True)

traingroup = folium.FeatureGroup(name='Train_Layer', control=True)


folium.GeoJson(
    geoJSON,
    style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(poagroup)

m.add_child(poagroup)


#m.save('../plots/poa_vic.html')

In [42]:
m = folium.Map(location=[-36, 145.3906], tiles="Stamen Terrain", zoom_start=7)

folium.GeoJson(
    geoJSON,
    style_function=lambda feature: {
        'fillColor': '#9ad7d2',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    },
    popup=folium.GeoJsonPopup(["Postcode"])
).add_to(poagroup)

m.add_child(poagroup)

HeatMap(pop_point).add_to(popgroup)
m.add_child(popgroup)

affgroup = folium.Choropleth(
    geo_data=re_inGeo, # geoJSON 
    name='Affordable_Layer', # name of plot
    data=re_in, # data source
    columns=['Postcode','affordable index'], # the columns required
    key_on= 'properties.Postcode', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Affordable Index'
)

affgroup.add_to(m)

marker_cluster_sch = MarkerCluster(name="School_Layer").add_to(schgroup)

# add data point to the mark cluster
for (lat, lng, types) in sch_point:
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='university',color='black',icon_color='white',angle=45),
        popup=types
    ).add_to(marker_cluster_sch)

# add marker_cluster to map
m.add_child(marker_cluster_sch)

marker_cluster_mall = MarkerCluster(name="Mall_Layer").add_to(mallgroup)

# add data point to the mark cluster
for (lat, lng, stores) in mall_point:
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='gift',color='orange',icon_color='white',prefix="fa"),
        popup=stores
    ).add_to(marker_cluster_mall)
    
m.add_child(marker_cluster_mall)
            
marker_cluster_train = MarkerCluster(name="Train_Layer").add_to(traingroup)

# add data point to the mark cluster
for (lat, lng, id) in train_point:
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='train',color='green',icon_color='white',prefix="fa"),
        popup=id
    ).add_to(marker_cluster_train)
    
m.add_child(marker_cluster_train)

folium.LayerControl().add_to(m)
m.save('../plots/poa_vic_mall.html')

marker_cluster = MarkerCluster().add_to(m)

# add data point to the mark cluster
for (lat, lng, stores) in mall_point:
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='gift',color='red',icon_color='blue',prefix="fa"),
        popup=stores
    ).add_to(marker_cluster)

for (lat, lng, types) in sch_point:
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='university',color='green',icon_color='yellow',prefix="fa"),
        popup=types
    ).add_to(marker_cluster)
    
for (lat, lng, id) in train_point:
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='train',color='purple',icon_color='yellow',prefix="fa"),
        popup=id
    ).add_to(marker_cluster)

# add marker_cluster to map
m.add_child(marker_cluster)
m.save('../plots/poa_vic_mall.html')

marker_cluster_sch = MarkerCluster().add_to(m)

# add data point to the mark cluster
for (lat, lng, types) in sch_point:
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='university',color='green',icon_color='yellow',angle=45),
        popup=types
    ).add_to(marker_cluster_sch)

# add marker_cluster to map
m.add_child(marker_cluster_sch)
m.save('../plots/poa_vic_sch.html')